In [1]:
import numpy as np
import pandas as pd

In [2]:
# principe
A = np.array([[1, 2, 3, 0, 1],[2, 3, 2, 2, 2],[0, 3, 3, 2, 2]])
cols = [[0,1], [0,3], [1,4]]
print('scenario array\n',A)
print('column combination to slice\n', cols)

scenario array
 [[1 2 3 0 1]
 [2 3 2 2 2]
 [0 3 3 2 2]]
column combination to slice
 [[0, 1], [0, 3], [1, 4]]


In [3]:
# slice column combination and compute max
A[:,cols].swapaxes(0,1).max(axis=2).T

array([[2, 1, 2],
       [3, 2, 3],
       [3, 2, 3]])

In [4]:
# load TableToArray numpy object
npy_path = r"D:\Projects\Pr\3728.10\Database\meren500.npy"
void = np.load(npy_path)

In [5]:
# prepare empty array
array = np.empty((void.shape[0], len(void[0])))
# parse void array
for row_ix in range(void.shape[0]):
    y = void[row_ix]
    array[row_ix] = np.array(y.tolist())   

In [6]:
# risk values are higher than 1, this is possible?
array[:,91]

array([1.10000002, 1.10000002, 1.10000002, ...,        nan,        nan,
              nan])

In [7]:
# set nan values to 0, multiply by ten-thousand and change type to integer
array[np.isnan(array)] = 0
array *= 10000
array = array.astype(int) 

In [8]:
# load event set
csv_path = r"D:\Projects\Pr\3728.10\Database\eventsetplus_v4_20190128\eventsetplus_Meren.csv"
df = pd.read_csv(csv_path)
df.head()

C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ID,Regio,T_Bin,DKR,Cond_Prob,Breach_ID1,Breach_name1,Flood_scenario_ID1,Flood_scenario_Name1,...,Breach_ID3,Breach_name3,Flood_scenario_ID3,Flood_scenario_Name3,Breach_ID4,Breach_name4,Flood_scenario_ID4,Flood_scenario_Name4,Flood_duration,Scenario_Prob
0,0,666,Meren,10000,6,0.006383,31,Lemmer Riensluis,165,geo_friesland_en_groningen__lemmer_riensluis__...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,5.744681e-07
1,1,667,Meren,100000,6,0.005769,31,Lemmer Riensluis,167,geo_friesland_en_groningen__lemmer_riensluis__...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,5.192308e-08
2,2,668,Meren,1000000,6,0.001408,31,Lemmer Riensluis,167,geo_friesland_en_groningen__lemmer_riensluis__...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,1.408451e-09
3,3,669,Meren,500,6,0.014286,32,Lemmer PM Sluis,168,geo_friesland_en_groningen__lemmer_pm_sluis__t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,1.428571e-05
4,4,670,Meren,1000,6,0.011765,32,Lemmer PM Sluis,168,geo_friesland_en_groningen__lemmer_pm_sluis__t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,5.882353e-06


In [9]:
# # get T500 events
# df_500 = df[df['T_Bin'] == 500]
# b = df_500[['Flood_scenario_ID1', 'Flood_scenario_ID2', 'Flood_scenario_ID3']]
# get all events
b = df[['Flood_scenario_ID1', 'Flood_scenario_ID2', 'Flood_scenario_ID3']]

enkelvoudig = b[(~b['Flood_scenario_ID1'].isnull()) & (b['Flood_scenario_ID2'].isnull()) & (b['Flood_scenario_ID3'].isnull())]
tweevoudig = b[(~b['Flood_scenario_ID1'].isnull()) & (~b['Flood_scenario_ID2'].isnull()) & (b['Flood_scenario_ID3'].isnull())]
drievoudig = b[(~b['Flood_scenario_ID1'].isnull()) & (~b['Flood_scenario_ID2'].isnull()) & (~b['Flood_scenario_ID3'].isnull())]

In [10]:
# parse tweevoudige breaches into array to be used as lookup on scenario dataframe
tweevoudig.loc[:, 'Flood_scenario_ID1'] = 'F' + tweevoudig['Flood_scenario_ID1'].astype(int).astype(str)
tweevoudig.loc[:, 'Flood_scenario_ID2'] = 'F' + tweevoudig['Flood_scenario_ID2'].astype(int).astype(str)
tweevoudig = tweevoudig[['Flood_scenario_ID1','Flood_scenario_ID2']].reset_index(drop=True)
tweevoudig.head()

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Flood_scenario_ID1,Flood_scenario_ID2
0,F168,F323
1,F168,F349
2,F168,F400
3,F168,F460
4,F168,F480


In [11]:
# load fieldnames which correspond to scenarios
path_columns = r"D:\Projects\Pr\3728.10\Database\fieldnames.csv"
s = pd.read_csv(path_columns)
s = s[2::4].reset_index(drop=True)
s.columns = ['id']
s.head()

,id
0,OBJECTID
1,nl_grid
2,X
3,Y
4,F81


In [12]:
# not all scenarios are present in the fieldnames.csv
# 1850 (-4) scenarios in fieldnames.csv
# 1930 (-1) scenarios in meren500.npy
s.shape, array.shape

((1850, 1), (69913, 1930))

In [13]:
# continue anyway
# get combinations as index array
combs = tweevoudig.values

idx_comb = []
for comb in combs:
    idxes = s.index[s['id'].isin(comb)].tolist()
    idx_comb.append(idxes)   

idx_comb = np.array(idx_comb)    
idx_comb

array([[  91,  246],
       [  91,  272],
       [  91,  323],
       ...,
       [1808, 1811],
       [1808, 1814],
       [1811, 1814]])

In [14]:
# number of combinations
idx_comb.shape

(5910, 2)

In [15]:
#%%timeit
# slice and compute max at once
selection = array[:, idx_comb].swapaxes(0,1).max(axis=2).T

In [16]:
selection / 10000

array([[1.1   , 1.1   , 1.1   , ..., 0.    , 0.    , 0.    ],
       [1.1   , 1.1   , 1.1   , ..., 0.    , 0.    , 0.    ],
       [1.1   , 1.1   , 1.1   , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.0049, 0.0049, 0.0049],
       [0.    , 0.    , 0.    , ..., 0.0074, 0.0074, 0.0074],
       [0.    , 0.    , 0.    , ..., 0.0199, 0.0199, 0.0199]])

In [17]:
# scenario risk are above 1, that seems not possible

In [18]:
# ~70.000 points for ~6.000 tweevoudige scenario combinations
selection.shape

(69913, 5910)